In [ ]:
import warnings
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta

from accure_io import PostgresInterface, S3Interface, SnowflakeInterface
from accure_io.s3_battery_data_reader import DataNotFoundError, S3BatteryDataReader
from accure_io._meta_data import MetaData
import accure_plot

In [ ]:
warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (15,6)

level = "pack"
customer = "senec"
pi = PostgresInterface()
dc = S3Interface.get_latest_data_context(customer=customer)
s3i = S3Interface(dc)

battery_reader = S3BatteryDataReader(tenant=customer, data_version="latest")
sfi = SnowflakeInterface(customer=customer)

health_path = "s3://accure-production-artifacts/senec/product=reivolution/data_version=2/run_context=submit-20220904/artifact_type=result/group=FCC_monthly/"
required_ids = 500


## Data pre-processiing

In [ ]:
version = "220928"
ids = pd.read_parquet(f"s3://accure-sandbox-data/kyung/{customer}/aging_accure_ids_{version}.parquet")

In [ ]:
id = ids["accure_id"][20]
meta = battery_reader.read_meta_data(level=level, accure_id=id)
time_start = meta.first_timestamp
time_end = meta.last_timestamp
ts_data = s3i.get_timeseries_s3(level=level, accure_id=id, time_start=time_start, time_end=time_end)
ts_data = ts_data[~ts_data.index.duplicated()]
invalid = ts_data["voltage"].isna() | ts_data["voltage"]==0 | ts_data["current"].isna()
ts_data = ts_data[~invalid]
display(ts_data)
nom_cap = meta.configurations.iloc[-1]["customer_datasheet"]['agg_capacity_design']
fcc_df = pd.read_parquet(f'{health_path}FCC_accure_id={id}.parquet')
soh = fcc_df['FCC_POINTS']/nom_cap
temp = ts_data['temperature2']
soc = ts_data['state_of_charge']
volt = ts_data['voltage']
curr = ts_data['current'] # positive = charge

### Visual checks

In [ ]:
# check first timestamps
id = ids["accure_id"][1]
meta = battery_reader.read_meta_data(level=level, accure_id=id)
time_start = meta.first_timestamp
time_start

In [ ]:
soh.plot()

In [ ]:
ts_data[["voltage","state_of_charge"]].plot.hist(rwidth=0.7)

In [ ]:
start = 0
end = 500
plt.subplot(3,1,1)
volt[start:end].plot(ylabel='volt')
plt.subplot(3,1,2)
soc[start:end].plot(ylabel='soc')
plt.subplot(3,1,3)
curr[start:end].plot(ylabel='curr')


In [ ]:
# verify no sudden jumps in voltage
print(sorted(volt.diff().abs(),reverse=True)[0:20])

In [ ]:
temp.plot(kind='hist',rwidth=0.7,edgecolor='black')

## Rainflow algo

In [ ]:
from accure_analytics.gaps.find_gaps import find_gaps
from accure_analytics.cycle_counting.rainflow import calculate_rainflow

# Import Datainterface
from accure_io.s3_battery_data_reader import S3BatteryDataReader

# Read the data
timeseries = battery_reader.read_sensor_data(level=level, accure_id=id)
timeseries

In [ ]:
from operator import contains
# timeseries = ts_data

timeseries = timeseries.sort_values("time")
timeseries.index = timeseries.index.tz_localize(None)
timeseries = timeseries[~timeseries.index.duplicated()]

invalid = timeseries["voltage"].isna() | timeseries["current"].isna()
timeseries = timeseries[~invalid]

timesteps = timeseries.index.to_series()
current = timeseries["current"].squeeze()
soc = timeseries["state_of_charge"].dropna().squeeze()

measurement_gaps = find_gaps(time_index=current.index)

import sys
sys.setrecursionlimit(10000)
rainflow = calculate_rainflow(soc=soc,current=current,gaps=measurement_gaps, idle_current_threshold_a=0)

In [ ]:
rainflow

## Visualizing change in SOH (d/dt)

In [ ]:
# forward diff
fd = soh.diff(periods=1).dropna()
# central diff
cd = soh.diff(periods=2).dropna()/2

plt.subplot(2,1,1)
fd.plot(xlim=[0,len(fd)],ylabel='Forward Diff')
plt.title('Rate of change in SOH')
plt.subplot(2,1,2)
cd.plot(xlim=[0,len(cd)],ylabel='Central Diff')


In [ ]:
temperature = []
voltage = []
current = []
for i in cd.index:
    date = time_start + relativedelta(months=i-1)
    start = f"{date.year}-{date.month}-01"
    end = pd.to_datetime(f"{date.year}-{date.month}-01")+relativedelta(months=1)
    end = f"{end.year}-{end.month}-01"
    data = ts_data[(ts_data.index>=start) & (ts_data.index<end)]
    temperature.append(data["temperature2"].mean())
    voltage.append(data["voltage"].mean())
    current.append(data["current"].abs().max())

dsoh = cd.reset_index()["FCC_POINTS"]
temperature = pd.Series(temperature)
voltage = pd.Series(voltage)
current = pd.Series(current)
# min-max normalization
dsoh = (dsoh-dsoh.min())/(dsoh.max()-dsoh.min())
temperature = (temperature-temperature.min())/(temperature.max()-temperature.min())
voltage = (voltage-voltage.min())/(voltage.max()-voltage.min())
current = (current-current.min())/(current.max()-current.min())
# mean normalization

temperature.plot(label='mean_temp')
voltage.plot(label='mean_volt')
current.plot(label='max_curr')
dsoh.plot(label='dSOH')
plt.legend()
plt.title('Normalized Comparison of indicators to change in health')


In [ ]:
cd = soh.diff(periods=2).dropna()/2
ind1 = []
ind2 = []
ind3 = []
for i in cd.index:
    # month range of FCC point
    date = time_start + relativedelta(months=i-1)
    start = f"{date.year}-{date.month}-01"
    end = pd.to_datetime(f"{date.year}-{date.month}-01")+relativedelta(months=1)
    end = f"{end.year}-{end.month}-01"
    data = ts_data[(ts_data.index>=start) & (ts_data.index<end)]
    rf = rainflow[(rainflow["time_start"]>=start)&(rainflow["time_end"]<end)]
    # indicators
    ind1.append(data["temperature2"].max()-data["temperature2"].min())
    ind2.append((rf["dod"]/rf["duration_h"]).sum())
    ind3.append(data["discharge_energy"].max()-data["discharge_energy"].min()+data["charge_energy"].max()-data["charge_energy"].min())

dsoh = cd.reset_index()["FCC_POINTS"]
ind1 = pd.Series(ind1)
ind2 = pd.Series(ind2)
ind3 = pd.Series(ind3)
# min-max normalization
dsoh = (dsoh-dsoh.min())/(dsoh.max()-dsoh.min())
ind1 = (ind1-ind1.min())/(ind1.max()-ind1.min())
ind2 = (ind2-ind2.min())/(ind2.max()-ind2.min())
ind3 = (ind3-ind3.min())/(ind3.max()-ind3.min())
# mean normalization

ind1.plot(label='temp_spread')
ind2.plot(label='dod/h')
ind3.plot(label='charge+discharge_energy')
dsoh.plot(label='dSOH')
plt.legend()
plt.title('Normalized Comparison of indicators to change in health')

## Collect aggregated datapoints for all ids

In [ ]:
cd = soh.diff(periods=2).dropna()/2
df = pd.DataFrame()
df["soh"] = soh
df["dsoh"] = cd.reset_index()["FCC_POINTS"]
for i in cd.index:
    # month range of FCC point
    date = time_start + relativedelta(months=i-1)
    start = f"{date.year}-{date.month}-01"
    end = pd.to_datetime(f"{date.year}-{date.month}-01")+relativedelta(months=1)
    end = f"{end.year}-{end.month}-01"
    data = ts_data[(ts_data.index>=start) & (ts_data.index<end)]
    rf = rainflow[(rainflow["time_start"]>=start)&(rainflow["time_end"]<end)]
    # indicators
    temperature.append(data["temperature2"].mean())
    voltage.append(data["voltage"].mean())
    current.append(data["current"].abs().max())
    ind1.append(data["temperature2"].max()-data["temperature2"].min())
    ind2.append((rf["dod"]/rf["duration_h"]).sum())
    ind3.append(data["discharge_energy"].max()-data["discharge_energy"].min()+data["charge_energy"].max()-data["charge_energy"].min())

ind1 = pd.Series(ind1)
ind2 = pd.Series(ind2)
ind3 = pd.Series(ind3)
# min-max normalization
dsoh = (dsoh-dsoh.min())/(dsoh.max()-dsoh.min())
ind1 = (ind1-ind1.min())/(ind1.max()-ind1.min())
ind2 = (ind2-ind2.min())/(ind2.max()-ind2.min())
ind3 = (ind3-ind3.min())/(ind3.max()-ind3.min())
# mean normalization

ind1.plot(label='temp_spread')
ind2.plot(label='dod/h')
ind3.plot(label='charge+discharge_energy')
dsoh.plot(label='dSOH')
plt.legend()
plt.title('Normalized Comparison of indicators to change in health')

## Internal Resistance

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

from accure_io import S3Interface
from accure_analytics.pipelines.internal_resistance.common import get_and_preprocess_timeseries, calc_df_for_cell_level
from accure_analytics.pipelines.utils.config_files.interface import get_task_config

from accure_analytics.internal_resistance.ri_soh_r import (
    calculate_ir_via_ohmic_law,
    calculate_ri_soh,
    describe_ri_soh,
    calculate_weekly_soh,
)

from accure_analytics.internal_resistance.lookup_table import (
    set_up_grid,
    read_ri_look_up_table,
    generate_all_systems_lookup,
    plot_ri_lookup_scatter,
    plot_ri_lookup_surface,
)

# for local regression:
from accure_analytics.utils.localreg import localreg 
from accure_analytics.utils.kernels import triweight

%load_ext autoreload
%autoreload 2

In [ ]:
customer = "senec"
project = "home-storage"
product = "internal_resistance"
level = "pack"
# device = "dcb102zk"
serial_con = 13  # for this device only
parallel_con = 1
accure_id = id  # this accure_id belongs to the type of device, and level etc. specified

In [ ]:
data_context = S3Interface.get_latest_data_context(customer = customer)
s3_interface = S3Interface(data_context)
df = get_and_preprocess_timeseries(s3_interface, level, accure_id)

In [ ]:
task_config = get_task_config(
    customer=customer, project=project, product=product, level=level
)

meta_obj = s3_interface.get_meta_s3(level=level, accure_id=accure_id)

df_cell = calc_df_for_cell_level(
    df=df,
    meta_obj=meta_obj,
    connection_meta_key=task_config["connection_meta_key"],
    serial_con=serial_con,
    parallel_con=parallel_con,
)
df_cell

In [ ]:
internal_resistance, _ = calculate_ir_via_ohmic_law(
    df_cell,
    current_diff_threshold_c=0.1,
)
